<a href="https://colab.research.google.com/github/x1001000/nss-aws-tests/blob/main/colab/UNet_video_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive') # 此處需要登入google帳號
# # 獲取授權碼之後輸入即可連動雲端硬碟
# data = pd.read_csv("/content/gdrive/My Drive/codev2/")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


需要新增存下影片檔名的code

 happen to have an existing .py file in Drive.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 18 17:07:35 2021
 
@author: lab70929
"""
 
from __future__ import print_function
import torch
import time
import os
import numpy as np
import matplotlib.pyplot as plt
from unet import UNet
import parameters
import cv2
import glob
from google.colab.patches import cv2_imshow
 
start_time = time.time()
device = torch.device("cuda")
 
def Unet_test(images, model, Size_X, Size_Y):
    batch_size = len(images)
    t = []
    t1 = time.time()
    inputImg = []
    inputImg_BK = []
    center_list = []
    for image in images:
        
        image = cv2.resize(image, (Size_X, Size_Y), interpolation=cv2.INTER_CUBIC)
        inputImg_BK.append(image.copy())  
        image = image.astype(np.float32)/255
        image = image[np.newaxis, :]
 
        inputImg.append(image)
    
    inputImg = np.array(inputImg)
        
    t2 = time.time()
    t.append(t2-t1)
    
    tf_images = torch.from_numpy(inputImg)
    tf_images = tf_images.to(device)
    output = model(tf_images)
    output_bk = output[:, 0].clone().detach().cpu().numpy()
    
    t3 = time.time()
    t.append(t3-t2)
    
    
    for i in range(batch_size):
        temp = output_bk[i]
        gt_temp = inputImg_BK[i]
        ttt = output_bk[i]
        ttt[ttt < 0.5] = 0
        ttt[ttt >= 0.5] = 1
        if np.count_nonzero(ttt) == 0:
            temp[temp < 0.25] = 0
            temp[temp >= 0.25] = 1
        else:
            temp[temp < 0.5] = 0
            temp[temp >= 0.5] = 1
    
        ## Connected Component Analysis
        if np.count_nonzero(temp) != 0:
            _, labels, stats, center = cv2.connectedComponentsWithStats(temp[:, :].astype(np.uint8))
    
            stats = stats[1:, :]
            pupil_candidate = np.argmax(stats[:, 4]) + 1
            temp[:, :][labels != pupil_candidate] = 0
            
        gt_temp[temp == 1] = 255
        output_bk[i] = temp
 
        index = np.argwhere(temp == 1)
        if index.shape != (0, 2):
            x_center = np.average(index[:,0])
            y_center = np.average(index[:,1])
            center_list.append([x_center, y_center])
        else:
            center_list.append([0, 0])
       
    t4 = time.time()
    t.append(t4-t3)
    
    return inputImg_BK, t, center_list
 
 
 
 
 
 
 
name_load_model = ("/content/gdrive/My Drive/codev2/trained_model/UNet/")
cross_val_num = 18
 
Size_X = parameters.Size_X
Size_Y = parameters.Size_Y
 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
 
 
model = UNet(n_channels=1, n_classes=1, bilinear=True)
if os.path.exists(name_load_model):
    load_saved_model_name = parameters.find_latest_model_name(name_load_model, cross_val_num)
    model.load_state_dict(torch.load(load_saved_model_name))
    print(parameters.C_GREEN + 'Check point Successfully Loaded' + parameters.C_END)
else:
    print(parameters.C_RED + 'Check point Not Found' + parameters.C_END)
 
 
model.eval()
model.to(device)
 
#left_roi = [[176,0],[540,300]]
#right_roi = [[750,0],[1130,340]]
left_roi = []
right_roi = []
 
video_folder = ("/content/gdrive/My Drive/codev2/Result/") 
# video_folder = "C:/Users/lab70929/Downloads/Debug_s7.2.5.1/Debug/Result/"
fps = []
video_list = glob.glob(video_folder+"*.mp4")
save_time = []
#for video_name in video_list:
video_name = video_list[0]
output_name = video_name[:-4] + "_tracking.mp4"
 
print("video_name = ", video_name)
 
out = cv2.VideoWriter(output_name, fourcc, 210, (384, 144), 0)
cap = cv2.VideoCapture(video_name)
frame_count = 0
 
 
batch_size = 16
counter = 0
images = []
temp_image = []
center_list = []
left_center_list_H = []
left_center_list_V = []
right_center_list_H = []
right_center_list_V = []
t = []
data = {'Left':{'Horizontal':{}, 'Vertical':{}}, 'Right':{'Horizontal':{}, 'Vertical':{}}, 'Timestamps':{}}
 
while (cap.isOpened()):
#    print(counter)
 
    
    
    if counter<batch_size//2:
        
        ret, frame = cap.read()
        if ret == False:
            break
    
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if left_roi ==[]:
            left_eye = frame[:, :640]
            right_eye = frame[:, 640:]
        else:
            left_eye = frame[left_roi[0][1]:left_roi[1][1], left_roi[0][0]:left_roi[1][0]]
            right_eye = frame[right_roi[0][1]:right_roi[1][1], right_roi[0][0]:right_roi[1][0]]
    
        images.append(left_eye)
        images.append(right_eye)
        counter += 1
 
    else:
        images, times, center_list = Unet_test(images, model, Size_X, Size_Y)
        save_time.append(times)
        for i in range(batch_size//2):
            left_result = images[i*2].astype(np.uint8)    
            right_result = images[i*2+1].astype(np.uint8)
            result = np.concatenate((left_result, right_result), axis = 1)
            out.write(result)
            # cv2_imshow(result)
            left_center_list = (center_list[i*2][0], center_list[i*2][1])
            right_center_list = (center_list[i*2+1][0], center_list[i*2+1][1])
 
            left_center_list_H.append(left_center_list[1])
            left_center_list_V.append(left_center_list[0])
            right_center_list_H.append(right_center_list[1])
            right_center_list_V.append(right_center_list[0])
            t.append(frame_count/2+i)
  
        frame_count +=batch_size
        images = []
        counter = 0
        data['Right'].update({'Horizontal': np.array(left_center_list_H)}) # this Right is for Right eye (at the left side)
        data['Right'].update({'Vertical': np.array(left_center_list_V)})  
        data['Left'].update({'Horizontal': np.array(right_center_list_H)}) # this Left is for Left eye (at the right side)
        data['Left'].update({'Vertical': np.array(right_center_list_V)})
        data.update({'Timestamps': np.array(t)})
 
        # print(data['Left']['Horizontal'])
 
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break       
    
end_time = time.time()
fps.append(frame_count / (end_time - start_time))
cap.release()
out.release()
cv2.destroyAllWindows()
 
output_video = cv2.VideoCapture(output_name)
length = int(output_video.get(cv2.CAP_PROP_FRAME_COUNT))
print("output frames = " , length )
 
 
cap = cv2.VideoCapture(output_name)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("input frames = " , length )
 
 
print("Total time: ", (end_time - start_time))

/content/gdrive/My Drive/codev2/trained_model/UNet/018/my_test_model_00057500iters.pt
Check point Successfully Loaded
video_name =  /content/gdrive/My Drive/codev2/Result/20210916_175947_H14_NSS16001_Test2.mp4
output frames =  18912
input frames =  18912
Total time:  580.7117583751678


In [ ]:
# 儲存變數到pkl檔案
import pickle

# Saving the objects:
with open(video_name[:-4]+'_unet_pixel_API.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(data, f)